_____

# Registration 

In [1]:
import os
import shutil
from itertools import combinations
from subprocess import Popen, PIPE
from multiprocessing import Pool, cpu_count

import numpy as np
import pandas as pd

In [2]:
def run_docker(cmd, container_name):  
    try:
        proc = Popen(cmd, shell=True, stdout=PIPE, stderr=PIPE)
        _ = proc.wait()
        if _!=0:
            print(proc.communicate())
            print(proc.stdout())
            return False
    except Exception as e:
        print (proc.communicate())
        print (e)
        proc_kill = Popen('docker stop '+ container_name, shell=True, stdout=PIPE)
        proc_kill.wait()
        proc_kill = Popen('docker rm ' + container_name, shell=True, stdout=PIPE)
        proc_kill.wait()
        proc.kill()
        return False
    return True

In [3]:
DOCKER_NAME = 'brain_pipeline/frsrf_minc_fsl'

def bet_brain_mask(docker_container_name, path_to_sourse, file_name):
    return '''docker run --rm \
    --name {1} \
    -v {2}:/input \
    {0} \
    bash -c ". /etc/fsl/5.0/fsl.sh \
    && bet /input/{3} /input/brainmask_{3} -m -n -f .4\
    " '''.format(DOCKER_NAME, # 0
                 docker_container_name, # 1
                 path_to_sourse, # 2
                 file_name # 3
                 )

In [4]:
def reg_command(img, toImg, patientName):
    imports='''export ANTSPATH=${HOME}/resnet_cnn_mri/ANTs/Scripts/
    export PATH=${ANTSPATH}:$PATH
    export ANTSPATH=${HOME}/resnet_cnn_mri/antsbin/bin
    export PATH=${ANTSPATH}:$PATH
    '''
    s = '''antsRegistrationSyNQuick.sh -d 3 -m {0} -f {1} -t s -o {2}'''
    return imports+s.format(img, toImg, patientName)

____-

In [5]:
HOME = '/home/mount/neuro-t01-ssd/home/amir/'
path_to_registration = os.path.join(HOME, 'projects/segm/projects/WMH/data/registration_for_histograms')
path_to_save = '/home/mount/neuro-x02-hdd/WMHS_AUGM/'

In [6]:
meta = pd.DataFrame()
university = []
path = []
path_to_mask = []
idx = []
path_br_mask = '../data/skull_stripping'
PATHS = ['../data/Utr/Utrecht/', '../data/Sing/Singapore/', '../data/Amst/GE3T/']

for PATH in PATHS:
    for i in os.listdir(PATH):
        university.append(PATH.split('/')[-2])
        idx.append(i)
        path_to_brain_mask = os.path.join(path_br_mask, PATH.split('/')[-2], i, 'brain_mask/brainmask_mask.nii.gz')
        path_for_wmhm = os.path.join(PATH, i, 'pre/FLAIR.nii.gz')
        path_for_wmhm = os.path.abspath(path_for_wmhm)
        path_to_mask.append(path_to_brain_mask)
        path.append(path_for_wmhm)
meta['idx'] = idx
meta["univ"] = university
meta['path_flair'] = path
meta['path_mask'] = path_to_mask

In [7]:
# сделали метадату со всеми путями

meta.sample(4)

,idx,univ,path_flair,path_mask
5,49,Utrecht,/home/mount/neuro-t01-ssd/home/amir/projects/s...,../data/skull_stripping/Utrecht/49/brain_mask/...
35,57,Singapore,/home/mount/neuro-t01-ssd/home/amir/projects/s...,../data/skull_stripping/Singapore/57/brain_mas...
53,132,GE3T,/home/mount/neuro-t01-ssd/home/amir/projects/s...,../data/skull_stripping/GE3T/132/brain_mask/br...
4,19,Utrecht,/home/mount/neuro-t01-ssd/home/amir/projects/s...,../data/skull_stripping/Utrecht/19/brain_mask/...


In [8]:
# список попарных регистраций
tmp_arr = []
for univ1, univ2 in combinations(meta.univ.unique(), 2):
    for first in meta[meta.univ==univ1].values:
        for second in meta[meta.univ==univ2].values:
            img_path, univ, idx, to_img_path, univ_to, idx_to=\
            first[2], first[1], first[0], second[2], second[1], second[0]
            tmp_arr.append((img_path, univ, idx, to_img_path, univ_to, idx_to))

In [9]:
len(tmp_arr)

1200

In [11]:
def registration_from_to(img_path, univ, idx, to_img_path, univ_to, idx_to):
    path = os.path.join(path_to_registration, univ+'_'+univ_to)
    if not os.path.exists(path):
        try:
            os.makedirs(path, mode=0o777)
        except Exception:
            pass
    path_to_save = os.path.join(path, idx+'_'+idx_to+'/')
    if not os.path.exists(path_to_save):
        try:
            os.makedirs(path_to_save, mode=0o777)
        except Exception:
            pass
    cmd = reg_command(img_path, to_img_path, path_to_save)
    print (cmd)
    try:
        proc = Popen(cmd, shell=True, stdout=PIPE, stderr=PIPE)
        a = proc.wait()
        if a!=0:
            print(proc.communicate())
            return False
    except Exception as e:
        print (proc.communicate())
        print (e)
    return path_to_save

In [12]:
def skull_strip_for(path_to_sourse, file_name):
    '''For new_registered_img'''
    docker_container_name = 'sk_str'+str(file_name)
    cmd = bet_brain_mask(docker_container_name, path_to_sourse, file_name)
    print(cmd)
    _ = run_docker(cmd, docker_container_name)
    return _

собрали список? зачем?

In [13]:
## Запуск на многих процессах;

In [45]:
def get_data_for_hist(meta):
    path = registration_from_to(*meta)
    if path:
        skull_strip_for(path+'/', 'Warped.nii.gz')
        skull_strip_for(path+'/', 'InverseWarped.nii.gz')
    p = np.array(meta)[[1,2,4,5]]
    p = '_'.join(p)
    print (path_to_save+p)
    shutil.copytree(path, path_to_save+p)
    shutil.rmtree(path)
    return True

In [46]:
tmp_arr[810:812]

[('/home/mount/neuro-t01-ssd/home/amir/projects/segm/projects/WMH/data/Sing/Singapore/68/pre/FLAIR.nii.gz',
  'Singapore',
  '68',
  '/home/mount/neuro-t01-ssd/home/amir/projects/segm/projects/WMH/data/Amst/GE3T/108/pre/FLAIR.nii.gz',
  'GE3T',
  '108'),
 ('/home/mount/neuro-t01-ssd/home/amir/projects/segm/projects/WMH/data/Sing/Singapore/68/pre/FLAIR.nii.gz',
  'Singapore',
  '68',
  '/home/mount/neuro-t01-ssd/home/amir/projects/segm/projects/WMH/data/Amst/GE3T/113/pre/FLAIR.nii.gz',
  'GE3T',
  '113')]

In [ ]:
with Pool(5) as pool: #cpu_count() - 2 or 1
    result = pool.map(get_data_for_hist, tmp_arr[0:50])
    print(np.sum(result))
    with open('output.txt', 'a') as f:
        f.write(str(sum(result)))
        f.write('\n')

export ANTSPATH=${HOME}/resnet_cnn_mri/ANTs/Scripts/
    export PATH=${ANTSPATH}:$PATH
    export ANTSPATH=${HOME}/resnet_cnn_mri/antsbin/bin
    export PATH=${ANTSPATH}:$PATH
    antsRegistrationSyNQuick.sh -d 3 -m /home/mount/neuro-t01-ssd/home/amir/projects/segm/projects/WMH/data/Utr/Utrecht/27/pre/FLAIR.nii.gz -f /home/mount/neuro-t01-ssd/home/amir/projects/segm/projects/WMH/data/Sing/Singapore/68/pre/FLAIR.nii.gz -t s -o /home/mount/neuro-t01-ssd/home/amir/projects/segm/projects/WMH/data/registration_for_histograms/Utrecht_Singapore/27_68/
export ANTSPATH=${HOME}/resnet_cnn_mri/ANTs/Scripts/
    export PATH=${ANTSPATH}:$PATH
    export ANTSPATH=${HOME}/resnet_cnn_mri/antsbin/bin
    export PATH=${ANTSPATH}:$PATH
    antsRegistrationSyNQuick.sh -d 3 -m /home/mount/neuro-t01-ssd/home/amir/projects/segm/projects/WMH/data/Utr/Utrecht/27/pre/FLAIR.nii.gz -f /home/mount/neuro-t01-ssd/home/amir/projects/segm/projects/WMH/data/Sing/Singapore/58/pre/FLAIR.nii.gz -t s -o /home/mount/neuro-t

In [ ]:
!ls $path_to_registration/

In [ ]:
ls /home/mount/neuro-x02-hdd/WMHS_AUGM/*

In [ ]:
!ls -la /home/mount/neuro-x02-hdd/WMHS_AUGM